**Многоэтапная сборка в docker**

В docker есть особый тип сборки, который позволяет оставить в финальном образе только те элементы, что нам нужны. Это, в некоторых случаях, позволит сэкономить размер docker образа.

# База

Формально это делается следующим образом:
```
FROM <название образа> AS <название сборки1>

FROM <название образа>
COPY --from=<название сборки1> <файл из перовой сборки> <файл из второй сборки>
```

### Пример

Так допуским ситуацию, где нам в финальной сборке надо иметь результаты обработки некоторого большого файла `example.csv` проводимой программой `generation.py` (в данном случае это просто взятие средних).

Если делать это **по простому**, то мы сделаем dockerfile вида (simple_dockerfile):

```
# простой dockerfile
FROM python:3.10 AS BUILD

WORKDIR some_dir
COPY example.csv example.csv
COPY requirements.txt requirements.txt
COPY generation.py generation.py

RUN python -m pip install --upgrade pip && pip install -r requirements.txt
RUN python generation.py

CMD ["cat", "means.csv"]
```

Далее:
- Собираю образ;
- Поднимаю контейнер на его основе, чтобы показать, что все работает номрмально;
- Показываю размер образа.

In [4]:
%%bash
docker build -q -t simple -f simple_dockerfile .
echo ==============================
docker run --rm simple
echo ==============================
docker images simple
echo ==============================
docker rmi simple

sha256:daea387ec3b5bb08725927e3cf8a8274d8c535fbdebd7905760ebeeb6aec0156
,0
0,0.49801512771615053
1,0.5005384992657755
2,0.5016440913126519
3,0.49893742832936905
4,0.5006416292879786
5,0.500604403582076
6,0.5024090926131595
7,0.500360289940759
8,0.4995687156844078
9,0.4993301862968266
10,0.4996320601371099
11,0.49936364688453744
12,0.4996892437983095
13,0.5003040557416716
14,0.5008896760787612
15,0.49834042413261304
16,0.5002565939707307
17,0.499441058229968
18,0.4994494362971878
19,0.5000038702075555
REPOSITORY   TAG       IMAGE ID       CREATED          SIZE
simple       latest    daea387ec3b5   43 minutes ago   1.11GB
Untagged: simple:latest
Deleted: sha256:daea387ec3b5bb08725927e3cf8a8274d8c535fbdebd7905760ebeeb6aec0156


В итоге мы вывели результат расчета. Ну и размер образа составляет титанические 1.11GB.

А теперь сделаем тоже самое, только **правильно**.  dockerfile в случае использоваиня многоэтапной сборке примет вид (multistage_dockerfile):

```
FROM python:3.10 AS BUILDER

WORKDIR some_dir
COPY example.csv example.csv
COPY requirements.txt requirements.txt
COPY generation.py generation.py

RUN python -m pip install --upgrade pip && pip install -r requirements.txt
RUN python generation.py

FROM ubuntu:22.04
WORKDIR some_dir
COPY --from=BUILDER /some_dir/means.csv /some_dir/means.csv
CMD ["cat", "means.csv"]
```

Буквально повторяю процедуры, которые проделал для предыдущего образа.

In [5]:
%%bash
docker build -q -t multistage -f multistage_dockerfile .
echo ==============================
docker run --rm multistage
echo ==============================
docker images multistage
echo ==============================
docker rmi multistage

sha256:15096f1d81baa6c46dadf24558dfa0c39ffcebb248495c779f1334b4b75a2eca
,0
0,0.49801512771615053
1,0.5005384992657755
2,0.5016440913126519
3,0.49893742832936905
4,0.5006416292879786
5,0.500604403582076
6,0.5024090926131595
7,0.500360289940759
8,0.4995687156844078
9,0.4993301862968266
10,0.4996320601371099
11,0.49936364688453744
12,0.4996892437983095
13,0.5003040557416716
14,0.5008896760787612
15,0.49834042413261304
16,0.5002565939707307
17,0.499441058229968
18,0.4994494362971878
19,0.5000038702075555
REPOSITORY   TAG       IMAGE ID       CREATED             SIZE
multistage   latest    15096f1d81ba   About an hour ago   77.8MB
Untagged: multistage:latest
Deleted: sha256:15096f1d81baa6c46dadf24558dfa0c39ffcebb248495c779f1334b4b75a2eca


И так, в результате, **все тоже самое** но размером в 77.8MB.

# Остановка на нужном этапе

При многоэтапной сборке, например, для отладки, может понадобиться отсновится в сборке образа. Это делается с помощью опции `--target <название этапа>`. Так, в примере далее, я останавливаю сборку описанного выше образа `multistage` на этапе `BUILDER` и убеждаюсь, что это именно тот этап (файлы то его).

In [8]:
%%bash
docker build -q -t multistage -f multistage_dockerfile --target BUILDER .
docker run --rm --name multistage -itd multistage
echo ==============================
docker exec multistage ls
echo ==============================
docker stop multistage
docker rmi multistage

sha256:19ec4683d03994e2f466696a18fbd3db8deda7f1f251185feddb1d568b116a02
0e9309e419e5c47647beb7005ecd0bb2d6c8c771eb96c216fcb24294781c4ff1
example.csv
generation.py
means.csv
requirements.txt
multistage
Untagged: multistage:latest
Deleted: sha256:19ec4683d03994e2f466696a18fbd3db8deda7f1f251185feddb1d568b116a02
